In [2]:
eventsPath = os.environ["YAHOO_DATA"]
rawEventsRdd = sc.textFile(eventsPath + "events.txt")
rawEventsRdd.take(3)

[u'001e6d8e-cbe7-4374-8c38-f37962a457e9\tair.com.smashatom.bingo\t1421009506\t47.237476\t-122.530884\tTacoma\t6\t12\tApp_Opened',
 u'001e6d8e-cbe7-4374-8c38-f37962a457e9\tcom.android.vending\t1421029924\t47.237476\t-122.530891\tTacoma\t6\t18\tApp_Opened',
 u'001e6d8e-cbe7-4374-8c38-f37962a457e9\tair.com.buffalo_studios.bingorush2\t1421015988\t47.237461\t-122.530899\tTacoma\t6\t14\tApp_Opened']

In [3]:
rawAppListRdd = sc.textFile(eventsPath + "userapplist.txt")
rawAppListRdd.take(3)

[u'001e6d8e-cbe7-4374-8c38-f37962a457e9\t{(com.android.dialer),(com.tct.soundrecorder),(com.android.systemui),(com.google.android.googlequicksearchbox),(com.android.email),(com.google.android.gm),(com.android.vending),(com.google.android.videos),(com.android.music),(com.jrdcom.torch),(com.android.calculator2),(com.android.phone),(com.playstudios.myvegas),(com.jrdcom.fmradio),(com.google.android.feedback),(com.android.browser),(com.google.android.music),(com.Qublix.SolitaireAtlantis),(com.facebook.orca),(com.android.chrome),(com.phonato.bingo.activities),(air.com.buffalo_studios.bingorush2),(com.offerup),(com.facebook.katana),(com.metropcs.service.vvm),(com.google.android.youtube),(com.jrdcom.android.gallery3d),(com.google.android.play.games),(com.google.android.talk),(com.teamlava.jewel),(com.google.android.videoeditor),(com.android.calendar),(com.google.android.apps.maps),(com.ea.BejeweledBlitz_na),(air.com.smashatom.bingo),(com.android.contacts),(com.android.mms),(com.android.setting

In [4]:
from collections import namedtuple

EventDataRow = namedtuple("EventDataRow", ["userId", "itemId", "ts", "latitude", "longitude", "city", "day_of_week", "time_of_day" , "event_type"])

def parseRawData(line):
    lineSplit = line.split("\t")
    return EventDataRow(userId=lineSplit[0],
                      itemId=lineSplit[1],
                      ts=int(lineSplit[2]),
                      latitude=float(lineSplit[3]),
                      longitude=float(lineSplit[4]),
                      city=lineSplit[5],
                      day_of_week=int(lineSplit[6]),
                      time_of_day=int(lineSplit[7]),
                      event_type=lineSplit[-1],
    )
    

eventsRdd = rawEventsRdd.map(parseRawData).cache()
eventsRdd.take(3)

[EventDataRow(userId=u'001e6d8e-cbe7-4374-8c38-f37962a457e9', itemId=u'air.com.smashatom.bingo', ts=1421009506, latitude=47.237476, longitude=-122.530884, city=u'Tacoma', day_of_week=6, time_of_day=12, event_type=u'App_Opened'),
 EventDataRow(userId=u'001e6d8e-cbe7-4374-8c38-f37962a457e9', itemId=u'com.android.vending', ts=1421029924, latitude=47.237476, longitude=-122.530891, city=u'Tacoma', day_of_week=6, time_of_day=18, event_type=u'App_Opened'),
 EventDataRow(userId=u'001e6d8e-cbe7-4374-8c38-f37962a457e9', itemId=u'air.com.buffalo_studios.bingorush2', ts=1421015988, latitude=47.237461, longitude=-122.530899, city=u'Tacoma', day_of_week=6, time_of_day=14, event_type=u'App_Opened')]

In [5]:
eventsRdd.filter(lambda x: x.city=="" ).take(10)

[EventDataRow(userId=u'03524163-5cdb-41a0-a983-a8332c00fe21', itemId=u'com.android.settings', ts=1421272442, latitude=42.344059, longitude=-71.097946, city=u'', day_of_week=2, time_of_day=16, event_type=u'App_Opened'),
 EventDataRow(userId=u'03524163-5cdb-41a0-a983-a8332c00fe21', itemId=u'com.umonistudio.tile', ts=1421260147, latitude=42.344036, longitude=-71.098083, city=u'', day_of_week=2, time_of_day=13, event_type=u'App_Opened'),
 EventDataRow(userId=u'03524163-5cdb-41a0-a983-a8332c00fe21', itemId=u'com.Splitwise.SplitwiseMobile', ts=1421065503, latitude=42.344025, longitude=-71.098038, city=u'', day_of_week=0, time_of_day=7, event_type=u'App_Opened'),
 EventDataRow(userId=u'03524163-5cdb-41a0-a983-a8332c00fe21', itemId=u'com.whatsapp', ts=1421326131, latitude=42.340755, longitude=-71.088097, city=u'', day_of_week=3, time_of_day=7, event_type=u'App_Opened'),
 EventDataRow(userId=u'03524163-5cdb-41a0-a983-a8332c00fe21', itemId=u'com.android.chrome', ts=1420980810, latitude=42.344002

In [7]:
userIdConversionDictionary = eventsRdd.map(lambda x: x.userId).distinct().zipWithIndex().collectAsMap()
userIdConversionDictionaryBroadcast = sc.broadcast(userIdConversionDictionary)
itemIdConversionDictionary = eventsRdd.map(lambda x: x.itemId).distinct().zipWithIndex().collectAsMap()
itemIdConversionDictionaryBroadcast = sc.broadcast(itemIdConversionDictionary)
cityConversionDictionary = eventsRdd.map(lambda x: x.city).distinct().zipWithIndex().collectAsMap()
cityConversionDictionaryBroadcast = sc.broadcast(cityConversionDictionary)

In [8]:
eventsConvertedRdd = eventsRdd.map(lambda x: EventDataRow(
    userId=userIdConversionDictionaryBroadcast.value[x.userId],
    itemId=itemIdConversionDictionaryBroadcast.value[x.itemId],
    ts=x.ts,
    latitude=x.latitude,
    longitude=x.longitude,
    city=cityConversionDictionaryBroadcast.value[x.city],
    day_of_week=x.day_of_week,
    time_of_day=x.time_of_day,
    event_type=x.event_type
    ))

eventsConvertedRdd.take(3)

[EventDataRow(userId=163385, itemId=77275, ts=1421009506, latitude=47.237476, longitude=-122.530884, city=13209, day_of_week=6, time_of_day=12, event_type=u'App_Opened'),
 EventDataRow(userId=163385, itemId=44053, ts=1421029924, latitude=47.237476, longitude=-122.530891, city=13209, day_of_week=6, time_of_day=18, event_type=u'App_Opened'),
 EventDataRow(userId=163385, itemId=83370, ts=1421015988, latitude=47.237461, longitude=-122.530899, city=13209, day_of_week=6, time_of_day=14, event_type=u'App_Opened')]

In [11]:
eventsConvertedRdd.filter(lambda eventRaw: eventRaw.event_type=='App_Opened').map(lambda eventRaw: (
    eventRaw.userId,eventRaw.itemId,eventRaw.ts,eventRaw.city,eventRaw.day_of_week,eventRaw.time_of_day,
    eventRaw.latitude,eventRaw.longitude)
        ).saveAsTextFile(eventsPath + "events_parsed")

In [ ]:
import json
with open(eventsPath + 'userIdConversionDictionary.txt', 'w') as outfile:
    json.dump(userIdConversionDictionary, outfile)
with open(eventsPath + 'itemIdConversionDictionary.txt', 'w') as outfile:
    json.dump(itemIdConversionDictionary, outfile)
with open(eventsPath + 'cityConversionDictionary.txt', 'w') as outfile:
    json.dump(cityConversionDictionary, outfile)

In [ ]:
def parseUserAppList(line):
    lineSplit = line.split("\t")
    return userIdConversionDictionary[lineSplit[0]],[itemIdConversionDictionary[app[1:-1]] for app in lineSplit[1][1:-1].split(",")]

appListRdd = rawAppListRdd.map(parseUserAppList)
appListMap = appListRdd.collectAsMap()
with open(eventsPath + '/userAppMap.txt', 'w') as outfile:
    json.dump(appListMap, outfile)
appListRdd.take(3)